# Ollama + OpenAI + Python

## 1. Укажите название модели

Если вы загрузили другую модель, отличную от "phi3:mini", измените значение в ячейке ниже. Эта переменная будет использоваться в коде на протяжении всего ноутбука.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Настройка клиента Open AI

Обычно клиент OpenAI используется с OpenAI.com или Azure OpenAI для взаимодействия с большими языковыми моделями.  
Однако его также можно использовать с Ollama, так как Ollama предоставляет совместимый с OpenAI конечный пункт по адресу "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Создание завершения чата

Теперь мы можем использовать OpenAI SDK для генерации ответа в рамках беседы. Этот запрос должен создать хайку о кошках:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Инженерия подсказок

Первое сообщение, отправленное языковой модели, называется "системным сообщением" или "системной подсказкой", и оно задает общие инструкции для модели.  
Вы можете задать свою собственную системную подсказку, чтобы направить языковую модель на генерацию ответа в другом стиле.  
Измените `SYSTEM_MESSAGE` ниже, чтобы модель отвечала, как ваш любимый персонаж из фильма или сериала, или получите вдохновение для других системных подсказок из [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

После того как вы настроите системное сообщение, задайте первый вопрос пользователя в `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Примеры с небольшим количеством данных

Еще один способ направить языковую модель — предоставить «несколько примеров», последовательность вопросов и ответов, которые демонстрируют, как она должна отвечать.

Пример ниже пытается заставить языковую модель вести себя как ассистент преподавателя, предоставляя несколько примеров вопросов и ответов, которые мог бы дать ассистент, а затем задает модели вопрос, который мог бы задать студент.

Попробуйте сначала, а затем измените `SYSTEM_MESSAGE`, `EXAMPLES` и `USER_MESSAGE` для нового сценария.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Генерация с использованием дополненного извлечения

RAG (Retrieval Augmented Generation) — это метод, позволяющий языковой модели точно отвечать на вопросы в конкретной области, сначала извлекая релевантную информацию из источника знаний, а затем генерируя ответ на основе этой информации.

Мы предоставили локальный CSV-файл с данными о гибридных автомобилях. Код ниже читает CSV-файл, ищет совпадения с вопросом пользователя и затем генерирует ответ на основе найденной информации. Обратите внимание, что выполнение этого процесса займет больше времени, чем предыдущие примеры, так как модели отправляется больше данных. Если вы заметите, что ответ все еще не основан на данных, вы можете попробовать инженерный подход к системе или использовать другие модели. В целом, RAG более эффективен с большими моделями или с тонко настроенными версиями SLM.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
